# Árboles de decisión: Churn compañía de servicios de telefonía móvil

Vamos a aplicar un modelo de clasificación de árboles de decisión a un dataset que describe los clientes de una compañía que presta servicios de telefonía móvil que se han abandonado o no la compañía para irse a la competencia.

Importamos las librerías que vamos a utilizar

In [6]:
import numpy as np #operaciones matriciales y con vectores
import pandas as pd #tratamiento de datos
import matplotlib.pyplot as plt #gráficos
from sklearn import tree, datasets, metrics
#from sklearn import neighbors, datasets, metrics
from sklearn.model_selection import train_test_split #metodo de particionamiento de datasets para evaluación
from sklearn.model_selection import cross_val_score, cross_validate #método para evaluar varios particionamientos de C-V
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, LeaveOneOut #Iteradores de C-V
import seaborn as sns
import math

## Entendimiento de los datos

Cargamos los datos para entenderlos

In [11]:
data = pd.read_csv("D:\OneDrive - Tecnoquimicas\99. PERSONAL\Formación\Maestria\Semestre 1\Fundamentos de Analitica I\08-06-churn.csv", sep=';', na_values=".")
print(data.shape)
data.head(5)

ValueError: embedded null character

**Parte 1**:
* Determine el número de registros, de variables, sus tipos ideales/reales, y sus rangos
* Determine el baseline y su accuracy. ¿Están balanceados los datos?
* ¿Encuentran algún problema con los datos (missing values, datos inválidos, etc.)?
* Si fuésemos a utilizar K-NN, ¿debería hacerse algún pretratamiento de los datos?

El método **describe** permite obtener un dataframe con una descripción de las variables de un dataframe analizado. Para cada variable encontramos el número de registros validos (*count*).

Además, si se trata de una variable categórica, se puede obtener el número de clases posibles (*unique*), la clase mayoritaria (*top*) y la frecuencia de la clase mayoritaria (*freq*).

Y, si se trata de una variable numérica, se puede obtener el promedio (*mean*), desviación estándar (*std*), los valores mínimos (*min*) y máximos (*max*) y los cuartiles (*25%*, *50%* y *75%*).

In [ ]:
data.describe(include="all")

In [ ]:
data.LEAVE.describe()

In [ ]:
data.LEAVE.describe()['freq'] / data.LEAVE.describe()['count']

In [ ]:
data.LEAVE.value_counts()

In [ ]:
data.LEAVE.value_counts().plot.bar()

.
.
.

RESPONDER AQUI A LAS PREGUNTAS DE LA PARTE 1.
.
.
.

**Parte 2**:

Utilizando pandas y matplotlib, analice la distribución de las variables independientes con respecto a los valores de la variable objetivo LEAVE y STAY. Trate de encontrar patrones en plots univariados (densidad) y Bivariados (scatterplots) para las variables numéricas, y gráficos de barras de conteo para las categóricas.

In [ ]:
var_indep_cat = ['COLLEGE', 'REPORTED_SATISFACTION', 'REPORTED_USAGE_LEVEL',
       'CONSIDERING_CHANGE_OF_PLAN', 'LEAVE']

In [ ]:
fig = plt.figure(figsize=(18,15))
i=1
for var_cat in var_indep_cat:
    ax = fig.add_subplot(math.ceil(len(var_indep_cat)/2), 2, i)
    sns.countplot(x=var_cat, hue="LEAVE", data=data)
    plt.title(var_cat)
    plt.legend(['LEAVE', 'STAY'])
    i+=1
    

In [ ]:
var_indep_cat.remove('LEAVE')

In [ ]:
var_indep_num = ['INCOME', 'OVERAGE', 'LEFTOVER', 'HOUSE', 'HANDSET_PRICE',
                 'OVER_15MINS_CALLS_PER_MONTH', 'AVERAGE_CALL_DURATION']

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fig = plt.figure(figsize=(18,15))
i=1
for var_num in var_indep_num:
    ax = fig.add_subplot(math.ceil(len(var_indep_num)/2), 2, i)
    sns.kdeplot(data[data['LEAVE']=='LEAVE'][var_num], shade=True, color='r', ax=ax);
    sns.kdeplot(data[data['LEAVE']=='STAY'][var_num], shade=True, color='g', ax=ax);
    plt.title(var_num)
    plt.legend(['LEAVE', 'STAY'])
    i+=1

In [ ]:
fig = plt.figure(figsize=(18,20))
ax = fig.add_subplot(3, 2, 1)
sns.scatterplot(x="HOUSE", y="INCOME", hue="LEAVE", data=data, ax=ax)
plt.title("HOUSE vs. INCOME")
ax = fig.add_subplot(3, 2, 2)
sns.scatterplot(x="HOUSE", y="OVERAGE", hue="LEAVE", data=data, ax=ax)
plt.title("HOUSE vs. OVERAGE")
ax = fig.add_subplot(3, 2, 3)
sns.scatterplot(x="HOUSE", y="LEFTOVER", hue="LEAVE", data=data, ax=ax)
plt.title("HOUSE vs. LEFTOVER")
ax = fig.add_subplot(3, 2, 4)
sns.scatterplot(x="HOUSE", y="HANDSET_PRICE", hue="LEAVE", data=data, ax=ax)
plt.title("HOUSE vs. HANDSET_PRICE")
ax = fig.add_subplot(3, 2, 5)
sns.scatterplot(x="HOUSE", y="OVER_15MINS_CALLS_PER_MONTH", hue="LEAVE", data=data, ax=ax)
plt.title("HOUSE vs. OVER_15MINS_CALLS_PER_MONTH")
ax = fig.add_subplot(3, 2, 6)
sns.scatterplot(x="HOUSE", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=data, ax=ax)
plt.title("HOUSE vs. AVERAGE_CALL_DURATION")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,20))
ax = fig.add_subplot(3, 2, 1)
sns.scatterplot(x="INCOME", y="OVERAGE", hue="LEAVE", data=data, ax=ax)
plt.title("INCOME vs. OVERAGE")
ax = fig.add_subplot(3, 2, 2)
sns.scatterplot(x="INCOME", y="LEFTOVER", hue="LEAVE", data=data, ax=ax)
plt.title("INCOME vs. LEFTOVER")
ax = fig.add_subplot(3, 2, 3)
sns.scatterplot(x="INCOME", y="HANDSET_PRICE", hue="LEAVE", data=data, ax=ax)
plt.title("INCOME vs. HANDSET_PRICE")
ax = fig.add_subplot(3, 2, 4)
sns.scatterplot(x="INCOME", y="OVER_15MINS_CALLS_PER_MONTH", hue="LEAVE", data=data, ax=ax)
plt.title("INCOME vs. OVER_15MINS_CALLS_PER_MONTH")
ax = fig.add_subplot(3, 2, 5)
sns.scatterplot(x="INCOME", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=data, ax=ax)
plt.title("INCOME vs. AVERAGE_CALL_DURATION")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,15))
ax = fig.add_subplot(2, 2, 1)
sns.scatterplot(x="OVERAGE", y="LEFTOVER", hue="LEAVE", data=data, ax=ax)
plt.title("OVERAGE vs. LEFTOVER")
ax = fig.add_subplot(2, 2, 2)
sns.scatterplot(x="OVERAGE", y="HANDSET_PRICE", hue="LEAVE", data=data, ax=ax)
plt.title("OVERAGE vs. HANDSET_PRICE")
ax = fig.add_subplot(2, 2, 3)
sns.scatterplot(x="OVERAGE", y="OVER_15MINS_CALLS_PER_MONTH", hue="LEAVE", data=data, ax=ax)
plt.title("OVERAGE vs. OVER_15MINS_CALLS_PER_MONTH")
ax = fig.add_subplot(2, 2, 4)
sns.scatterplot(x="OVERAGE", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=data, ax=ax)
plt.title("OVERAGE vs. AVERAGE_CALL_DURATION")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,15))
ax = fig.add_subplot(2, 2, 1)
sns.scatterplot(x="LEFTOVER", y="HANDSET_PRICE", hue="LEAVE", data=data, ax=ax)
plt.title("LEFTOVER vs. HANDSET_PRICE")
ax = fig.add_subplot(2, 2, 2)
sns.scatterplot(x="LEFTOVER", y="OVER_15MINS_CALLS_PER_MONTH", hue="LEAVE", data=data, ax=ax)
plt.title("LEFTOVER vs. OVER_15MINS_CALLS_PER_MONTH")
ax = fig.add_subplot(2, 2, 3)
sns.scatterplot(x="LEFTOVER", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=data, ax=ax)
plt.title("LEFTOVER vs. AVERAGE_CALL_DURATION")
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,5))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(x="HANDSET_PRICE", y="OVER_15MINS_CALLS_PER_MONTH", hue="LEAVE", data=data, ax=ax)
plt.title("HANDSET_PRICE vs. OVER_15MINS_CALLS_PER_MONTH")
ax = fig.add_subplot(1, 2, 2)
sns.scatterplot(x="HANDSET_PRICE", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=data, ax=ax)
plt.title("HANDSET_PRICE vs. AVERAGE_CALL_DURATION")
plt.show()

Algunos de los plots no son fáciles de interpretar porque los puntos están uno encima de otro.
Para estos casos se puede agregar un poco de ruido (**jitter**) a los puntos. 
Vamos a agregarlo a mano para el siguiente plot (increiblemente Python no tiene esta funcionalidad!!!).
Veamos como sería sin el jitter.

In [ ]:
fig = plt.figure(figsize=(18,5))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(x="OVER_15MINS_CALLS_PER_MONTH", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=data, ax=ax)
plt.title("OVER_15MINS_CALLS_PER_MONTH vs. AVERAGE_CALL_DURATION")
plt.show()

Ahora veamos con el jitter (creado a mano), combinado con una reducción en el tamaño de los puntos para que se vea una mayor cantidad de puntos.

In [ ]:
d2 = data[["OVER_15MINS_CALLS_PER_MONTH", "AVERAGE_CALL_DURATION", "LEAVE"]]
jitter = 0.3
d2.OVER_15MINS_CALLS_PER_MONTH = data.OVER_15MINS_CALLS_PER_MONTH + np.random.normal(scale=jitter, size=20000)
d2.AVERAGE_CALL_DURATION = data.AVERAGE_CALL_DURATION + np.random.normal(scale=jitter, size=20000)

In [ ]:
fig = plt.figure(figsize=(18,5))
ax = fig.add_subplot(1, 2, 1)
sns.scatterplot(x="OVER_15MINS_CALLS_PER_MONTH", y="AVERAGE_CALL_DURATION", hue="LEAVE", data=d2, ax=ax, size=1)
plt.title("OVER_15MINS_CALLS_PER_MONTH vs. AVERAGE_CALL_DURATION")
plt.show()

# Clasificación a partir un árbol de decisión

Los árboles de decisión permiten utilizar tanto las variables predictivas categóricas como las numéricas.
No todas las variables van a ser útiles. El árbol se va a encargar de encontrar la mejor variable a utilizar en el contexto del subconjunto de datos de cada rama.

Veamos cómo se crea un árbol de decisión en scikit-learn, y algunos de los parámetros más importantes:

In [ ]:
np.random.seed(1234)
ctree = tree.DecisionTreeClassifier(
    criterion='entropy', # el criterio de particionamiento de un conjunto de datos (gini, entropy)
    max_depth=None,      # prepoda: controla la profundidad del árbol (largo máximo de las ramas)
    min_samples_split=2, # prepoda: el mínimo número de registros necesarios para crear una nueva rama
    min_samples_leaf=1,  # prepoda: el mínimo número de registros en una hoja
    random_state=None,   # semilla del generador aleatorio utilizado para 
    max_leaf_nodes=None, # prepoda: máximo número de nodos hojas
    min_impurity_decrease=0.0, # prepoda: umbral mínimo de reducción de la impureza para aceptar la creación de una rama
    class_weight=None    # permite asociar pesos a las clases, en el caso de diferencias de importancia entre ellas
)

**Nota editorial**: scikit-learn soporta en sus árboles de decisión solamente variables independientes numéricas!!!!! Toca entonces utilizar un encoding (one hot).

In [ ]:
data_cat_one_hot = pd.get_dummies(data[var_indep_cat], prefix=var_indep_cat)
X = data[var_indep_num].join(data_cat_one_hot)
y = data['LEAVE']
X.shape

Quedaron 24 variables independientes.

In [ ]:
np.random.seed(1234)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [ ]:
ctree.fit(X_train, y_train)

Una vez el árbol es aprendido, se puede consultar diferentes atributos.
El mas interesante, aparte del árbol en sí, es el que asocia un índice de importancia a los atributos independientes en la clasificación:

In [ ]:
ctree.feature_importances_

In [ ]:
X.columns

Encontramos entonces que las variables en orden de importancia son:

In [ ]:
X.columns[np.argsort(-ctree.feature_importances_)]

Vamos ahora a visualizar el árbol aprendido (para hacerlo, se necesita preinstalar la aplicación graphviz).

In [ ]:
import pydotplus
from IPython.display import Image  
from sklearn.externals.six import StringIO  

In [ ]:
dot_data = StringIO()
tree.export_graphviz(ctree, 
                     filled=True, rounded=True,  #nodos redondeados y coloreados
                     class_names=ctree.classes_,
                     feature_names=X_train.columns,  
                     out_file=dot_data,
                     special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

La imágen es demasiado grande para poder visualizarla aquí, vamos a guardarla un archivo y abrirla en un visor externo que permita hacer zoom.

In [ ]:
# Create PNG
graph.write_png("arbol.png")

Como se puede ver el árbol es increiblemente grande y complejo, pues no se especificó ninguna manera de limitar su crecimiento.

## Evaluación del modelo

Ya tenemos el modelo "aprendido" con el dataset de 20000 instancias.
Vamos ahora a evaluarlo sobre ese mismo dataset para poder ver los éxitos y errores de la predicción. 

In [ ]:
y_pred = ctree.predict(X_test)

In [ ]:
cm= metrics.confusion_matrix(y_test, y_pred)
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title("Matriz de confusión para K=5")
plt.colorbar()
tick_marks = np.arange(2)
plt.xticks(tick_marks, y_test.unique())
plt.yticks(tick_marks, y_test.unique())
plt.show()

In [ ]:
print(cm)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_pred))
print("Precisión     : ", metrics.precision_score(y_test, y_pred, "LEAVE", average='macro'))
print("Recall        : ", metrics.recall_score(y_test, y_pred, "LEAVE", average='macro'))
VN = np.sum(cm[1:3,1:3])
FP = np.sum(cm[0,1:3])
specificity = VN/(VN+FP)
print("Especificidad : ", specificity)
print("F1-score      : ", metrics.f1_score(y_test, y_pred, "LEAVE", average='macro'))


Con el árbol completo, obtuvimos un nivel de accuracy del 61.68%, cuando el baseline era de 50.74%.

## Overfitting: poda del árbol

Vamos a ver que hubiera pasado si limitamos el crecimiento del árbol (prepoda), controlando la profundidad del árbol y el mínimo número de registros de un nodo para permitir el particionamiento.

**Parte 3**:

Modifiquen el parámetro **criterion** y los parámetros de pre-poda buscando una mejor exactitud del modelo:
- **max_depth**: entre mas grande el valor, el árbol será más complejo (más número de niveles de profundidad)
- **min_samples_split**: entre mas grande el valor, el árbol será más sencillo (se necesita tener más registros en un nodo para poder particionarlo)
- **min_samples_leaf**: entre mas grande el valor, el árbol será más sencillo (se necesita tener más registros en una hoja para poder aceptarla, si no se llega a esa cardinalidad, no se permite el partionamiento de su nodo padre) 
- **min_impurity_decrease**: entre mas grande el valor, el árbol será más sencillo (un nivel de impureza bajo inferior a este umbral no desatará un particionamiento. 


In [ ]:
np.random.seed(1234)
ctree = tree.DecisionTreeClassifier(
    criterion='entropy', # el criterio de particionamiento de un conjunto de datos
#    max_depth=,      # prepoda: controla la profundidad del árbol (largo máximo de las ramas)
#    min_samples_split=, # prepoda: el mínimo número de registros necesarios para crear una nueva rama
#    min_samples_leaf=,  # prepoda: el mínimo número de registros en una hoja
    random_state=None,   # semilla del generador aleatorio utilizado para 
#    max_leaf_nodes=, # prepoda: máximo número de nodos hojas
    min_impurity_decrease=0.0, # prepoda: umbral mínimo de reducción de la impureza para aceptar la creación de una rama
    class_weight=None    # permite asociar pesos a las clases, en el caso de diferencias de importancia entre ellas
)
ctree.fit(X_train, y_train)

In [ ]:
y_pred = ctree.predict(X_test)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))

In [ ]:
dot_data = StringIO()
tree.export_graphviz(ctree, 
                     filled=True, rounded=True,  #nodos redondeados y coloreados
                     class_names=ctree.classes_,
                     feature_names=X_train.columns,  
                     out_file=dot_data,
                     special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

Podríamos seguir mejorando el árbol buscando un tuning de la prepoda mas complejo con los demás parámetros.

Podemos intentar con otro tipo de modelos, e.g KNN.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(1234)
knn = KNeighborsClassifier(n_neighbors=300)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
cm= metrics.confusion_matrix(y_test, y_pred)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_pred))

También con un modelo Naive Bayes 

In [ ]:
from sklearn.naive_bayes import GaussianNB
np.random.seed(1234)
nb = GaussianNB()
nb.fit(X_train, y_train)
y_pred = nb.predict(X_test)
cm= metrics.confusion_matrix(y_test, y_pred)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_pred))

## Modelos de ensamble: bagging, random forest, boosting

## Bagging

Vamos ahora a crear un modelo de ensamble que utiliza muchos modelos de árboles sencillos que pone a votar para encontrar una decisión consensuada.

In [ ]:
from sklearn.ensemble import BaggingClassifier
np.random.seed(1234)
ctree = tree.DecisionTreeClassifier(
    criterion='entropy', # el criterio de particionamiento de un conjunto de datos
    max_depth=7,      # prepoda: controla la profundidad del árbol (largo máximo de las ramas)
    min_samples_split=1000, # prepoda: el mínimo número de registros necesarios para crear una nueva rama
    min_samples_leaf=1,  # prepoda: el mínimo número de registros en una hoja
    random_state=None,   # semilla del generador aleatorio utilizado para 
    max_leaf_nodes=None, # prepoda: máximo número de nodos hojas
    min_impurity_decrease=0.0, # prepoda: umbral mínimo de reducción de la impureza para aceptar la creación de una rama
    class_weight=None    # permite asociar pesos a las clases, en el caso de diferencias de importancia entre ellas
)
bagging = BaggingClassifier(base_estimator = ctree,   # Por defecto un decision tree 
                            n_estimators=200,          # Número de modelos a crear
                            max_samples=0.7,          # Número o % de registros de la muestra de aprendizaje
                            max_features=0.7,         # Número o % de atributos de la muestra de aprendizaje
                            bootstrap=True,           # Utilizar reemplazo en el muestreo de los registros de aprendizaje
                            bootstrap_features=False, # Utilizar reemplazo en el muestreo de los atributos de aprendizaje
                            oob_score=False,          # Evaluar cada modelo con los registros no utilizados en su aprendizaje 
                            n_jobs=2,                 # Número de cores a utilizar
                            random_state=None,        # random seed para el generador aleatorio
                            verbose=0)                # controla la cantidad de información a reportar


In [ ]:
bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
cm= metrics.confusion_matrix(y_test, y_pred)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_pred))

Podemos hacer bagging de otro tipo de estimadores de base, por ejemplo KNN.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
np.random.seed(1234)
bagging = BaggingClassifier(base_estimator = KNeighborsClassifier(),   # Por defecto un decision tree 
                            n_estimators=50,          # Número de modelos a crear
                            max_samples=0.5,          # Número o % de registros de la muestra de aprendizaje
                            max_features=0.5,         # Número o % de atributos de la muestra de aprendizaje
                            bootstrap=True,           # Utilizar reemplazo en el muestreo de los registros de aprendizaje
                            bootstrap_features=False, # Utilizar reemplazo en el muestreo de los atributos de aprendizaje
                            oob_score=False,          # Evaluar cada modelo con los registros no utilizados en su aprendizaje 
                            n_jobs=2,                 # Número de cores a utilizar
                            random_state=None,        # random seed para el generador aleatorio
                            verbose=0)                # controla la cantidad de información a reportar

bagging.fit(X_train, y_train)
y_pred = bagging.predict(X_test)
cm= metrics.confusion_matrix(y_test, y_pred)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_pred))

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
np.random.seed(1234)
rforest = RandomForestClassifier(n_estimators=100,     #Número de modelos a crear 
                                criterion='entropy', # el criterio de particionamiento de un conjunto de datos
                                max_depth=6,      # prepoda: controla la profundidad del árbol (largo máximo de las ramas)
                                min_samples_split=1000, # prepoda: el mínimo número de registros necesarios para crear una nueva rama
                                min_samples_leaf=1,  # prepoda: el mínimo número de registros en una hoja
                                max_features=('auto'), # Número o % de atributos de la muestra de aprendizaje
                                max_leaf_nodes=None, # prepoda: máximo número de nodos hojas
                                min_impurity_decrease=0.0, # prepoda: umbral mínimo de reducción de la impureza para aceptar la creación de una rama
                                bootstrap=True,      # Utilizar reemplazo en el muestreo de los registros de aprendizaje
                                oob_score=True,     # Evaluar cada modelo con los registros no utilizados en su aprendizaje 
                                n_jobs=2,            # Número de cores a utilizar
                                random_state=None,   # random seed para el generador aleatorio
                                verbose=0)           # controla la cantidad de información a reportar
rforest.fit(X_train, y_train)
y_pred = rforest.predict(X_test)
cm= metrics.confusion_matrix(y_test, y_pred)
print("Exactitud: ", metrics.accuracy_score(y_test, y_pred))
print("Kappa    : ", metrics.cohen_kappa_score(y_test, y_pred))

In [ ]:
rforest.oob_score_

Encontramos que las variables en orden de importancia son:

In [ ]:
X.columns[np.argsort(-rforest.feature_importances_)]

## Boosting

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import zero_one_loss
np.random.seed(1234)

numModelos = 400
ada_10 = AdaBoostClassifier(base_estimator=None,  #Por defecto se trata de decision stumps
                         n_estimators=numModelos,     #Número de modelos a crear
                         algorithm='SAMME',
                         learning_rate=1.0)    #Reduce la importancia de los modelos mas recientes
ada_10.fit(X_train, y_train)
ada_02 = AdaBoostClassifier(base_estimator=None,  #Por defecto se trata de decision stumps
                         n_estimators=numModelos,     #Número de modelos a crear
                        algorithm='SAMME',
                         learning_rate=0.2)    #Reduce la importancia de los modelos mas recientes

ada_02.fit(X_train, y_train)
ada_005 = AdaBoostClassifier(base_estimator=None,  #Por defecto se trata de decision stumps
                         n_estimators=numModelos,     #Número de modelos a crear
                         algorithm='SAMME',
                         learning_rate=0.05)    #Reduce la importancia de los modelos mas recientes
ada_005.fit(X_train, y_train)

In [ ]:
ada_errores_test_10 = np.zeros((numModelos,))
for i, y_pred in enumerate(ada_10.staged_predict(X_test)):
    ada_errores_test_10[i] = 1-zero_one_loss(y_pred, y_test)
ada_errores_train_10 = np.zeros((numModelos,))
for i, y_pred in enumerate(ada_10.staged_predict(X_train)):
    ada_errores_train_10[i] = 1-zero_one_loss(y_pred, y_train)

In [ ]:
ada_errores_test_02 = np.zeros((numModelos,))
for i, y_pred in enumerate(ada_02.staged_predict(X_test)):
    ada_errores_test_02[i] = 1-zero_one_loss(y_pred, y_test)
ada_errores_train_02 = np.zeros((numModelos,))
for i, y_pred in enumerate(ada_02.staged_predict(X_train)):
    ada_errores_train_02[i] = 1-zero_one_loss(y_pred, y_train)

In [ ]:
ada_errores_test_005 = np.zeros((numModelos,))
for i, y_pred in enumerate(ada_005.staged_predict(X_test)):
    ada_errores_test_005[i] = 1-zero_one_loss(y_pred, y_test)

ada_errores_train_005 = np.zeros((numModelos,))
for i, y_pred in enumerate(ada_005.staged_predict(X_train)):
    ada_errores_train_005[i] = 1-zero_one_loss(y_pred, y_train)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(np.arange(numModelos) + 1, ada_errores_test_10,
        label='AdaBoost Test Acc lr 1.0',
        color='red')
ax.plot(np.arange(numModelos) + 1, ada_errores_train_10,
        label=' AdaBoost Train Acc lr 1.0',
        color='blue')
ax.plot(np.arange(numModelos) + 1, ada_errores_test_02,
        label='AdaBoost Test Acc lr 0.2',
        color='gray')
ax.plot(np.arange(numModelos) + 1, ada_errores_train_02,
        label=' AdaBoost Train Acc lr 0.2',
        color='cyan')
ax.plot(np.arange(numModelos) + 1, ada_errores_test_005,
        label='AdaBoost Test Acc lr 0.05',
        color='green')
ax.plot(np.arange(numModelos) + 1, ada_errores_train_005,
        label=' AdaBoost Train Acc lr 0.05',
        color='yellow')

ax.set_ylim((0.6, 0.7))
ax.set_xlabel('# de modelos')
ax.set_ylabel('acc')

leg = ax.legend(loc='lower right', fancybox=True)
leg.get_frame().set_alpha(0.7)

plt.show()